# STT -> LLM -> TTS Test Space

pipeline and stack:

* STT: coqui tts, vosk (which one?)
* LLM: ollama, langchain
* TTS: coqui tts
* AUDIO I/O: pyaudio, sounddevice

### Audio I/O Testing

In [2]:
# audio IO - pyaudio test (playback - sample)
import wave
import sys
import pyaudio

chunksize = 1024
f = 'output.wav'

with wave.open(f, 'rb') as wf:
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()

    # Open steam (2)
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Play samples from the wave file (3)
    while len(data := wf.readframes(chunksize)):
        stream.write(data)

    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()

In [3]:
# audio IO - pyaudio test (record)
import wave
import sys
import pyaudio
import math

chunksize = 1024
f = 'record.wav'
seconds = 5
rate = 44100
channels = 1
form = pyaudio.paInt16

# Instantiate PyAudio and initialize PortAudio system resources (1)
p = pyaudio.PyAudio()

# Open steam (2)
stream = p.open(format=form,
                channels=channels,
                rate=rate,
                input=True,
                frames_per_buffer=chunksize)

# instantiate frames container
print ("recording started")
recordframes = []

# record w/ logic for seconds
for i in range(0, math.ceil(rate / chunksize * seconds)):
    data = stream.read(chunksize)
    recordframes.append(data)
print ("recording stopped")
stream.stop_stream()

# Close stream (4)
stream.close()

# Release PortAudio system resources (5)
p.terminate()

# wave file
wf = wave.open(f, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(form))
wf.setframerate(rate)
wf.writeframes(b''.join(recordframes))
wf.close()

recording started
recording stopped


In [4]:
# audio IO - pyaudio test (playback - sample)
import wave
import sys
import pyaudio

chunksize = 1024
f = 'record.wav'

with wave.open(f, 'rb') as wf:
    # Instantiate PyAudio and initialize PortAudio system resources (1)
    p = pyaudio.PyAudio()

    # Open steam (2)
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # Play samples from the wave file (3)
    while len(data := wf.readframes(chunksize)):
        stream.write(data)

    # Close stream (4)
    stream.close()

    # Release PortAudio system resources (5)
    p.terminate()

### Voice Synthesis Testing

In [5]:
import torch
from TTS.api import TTS
from datetime import date 

script = 'Hey fryman, pass me the peanut butter'

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Text to speech to a file
tts.tts_to_file(text=script, speaker_wav="wav_training/p1.wav", language="en", file_path=f"wav_sample/test_p1_{date.today().strftime('%Y%m%d%H%M%S')}.wav")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['Hey fryman, pass me the peanut butter']
 > Processing time: 2.366482734680176
 > Real-time factor: 0.6703615660290067


'wav_sample/test_p1_20250319000000.wav'

### LLM Instantiation Testing